<h1>Overpass API Notebook</h1>

<p>This notebook is dedicated to call the overpass API and extract data for our ADS Project. The data will enrich our rental price prediction models with relevant geo-spacial data_ in the vicitnity of a given apartment such as restaurants, supermarkets and other points of interest.<p> 

In [10]:
import json
import requests
import pandas as pd


# Request all restaurants, cafes, and supermarkets in the Kanton of Zurich, Switzerland
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["alt_name"="Kanton Zürich"]->.a;
(node["amenity"="restaurant"](area.a);
 node["amenity"="cafe"](area.a);
 node["shop"="supermarket"](area.a););
out center;
"""


# Server response
response = requests.get(overpass_url, params={'data': overpass_query})

data = response.json()

# Extract relevant data from the JSON response
places = []
for element in data['elements']:
    if 'tags' in element:
        place_type = element['tags'].get('amenity') or element['tags'].get('shop')
        places.append({
            'id': element['id'],
            'type': place_type,
            'lat': element['lat'],
            'lon': element['lon'],
            'name': element['tags'].get('name', ''),
            'brand': element['tags'].get('brand', ''),
            'address_city': element['tags'].get('addr:city', ''),
            'address_zip': element['tags'].get('addr:postcode', ''),
            'address_street': element['tags'].get('addr:street', ''),
            'address_number': element['tags'].get('addr:housenumber', ''),
            'website': element['tags'].get('website', '')
            

        })

# Convert the list of dictionaries to a pandas DataFrame
places_df = pd.DataFrame(places)

# Display the DataFrame
places_df


,id,type,lat,lon,name,brand,address_city,address_zip,address_street,address_number,website
0,34039212,restaurant,47.495518,8.741556,Hallenbadrestaurant Geiselweid,,,,,,
1,34040796,restaurant,47.483065,8.705704,Schwimmbadrestaurant Töss,,,,,,
2,39947904,supermarket,47.376732,8.542161,Coop,Coop,Zürich,8001,Bahnhofbrücke,1,
3,48932835,supermarket,47.375020,8.522895,Migros,Migros,Zürich,8004,Wengistrasse,7,https://filialen.migros.ch/de/migros-supermark...
4,57370928,restaurant,47.245360,8.726042,Metzg,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...
3566,11755283869,cafe,47.358640,8.523193,Cake Friends,,,,,,
3567,11764916519,restaurant,47.319772,8.521702,Restaurant Soodhof,,,,,,
3568,11775729223,cafe,47.374081,8.531948,Bang On,,Zürich,8004,Kasernenstrasse,15,https://www.bang-on.ch/
3569,11780149080,restaurant,47.418242,8.555513,,,,,,,


In [11]:
types = ['restaurant', 'cafe', 'supermarket']

for i in types:
    print(i ,':', (places_df['type'] == i).sum())

restaurant : 2337
cafe : 608
supermarket : 624


In [1]:
import pandas as pd

filepath = '../TablesDB/Location_v3.csv'

location_df = pd.read_csv(filepath, delimiter=",")
location_df

,LocationId,Street,ZIP,longitude,latitude
0,78753131-6d78-4d1e-a0e5-fc8b5f430570,NaN,6598.0,8.849665,46.175962
1,f0c2bc0e-55ab-4eb1-98fa-edcb4a0ee01e,via albonago 43,6962.0,8.974113,46.013360
2,fbdcde66-0fd7-4304-a83e-70ecbb0f6ae7,Via San Gottardo 41,6500.0,8.808332,46.174824
3,2d7451a3-d7e1-4dc7-babf-ab237f2c02e4,Via F. Zorzi 17,6500.0,9.015443,46.187900
4,2bde1434-f480-4d46-9b05-99e0c0659671,Via San Gottardo,6900.0,8.840817,46.178948
...,...,...,...,...,...
11347,2dba2402-d23f-4ab8-9bb6-702bbe28cb4e,Chemin de la Petite-Californie,1222.0,6.207831,46.242178
11348,daed4779-995c-4c34-85e7-4b269d8bf5cc,Chemin William-Barbey,1292.0,6.149006,46.241745
11349,faddd38c-e9fc-4f4d-bf09-a41818cc620f,Rue du Temple 5,1236.0,6.018499,46.174626
11350,84d332ee-e1a8-4b5f-a432-82819fffe330,Chemin de la Pierre-à-Bochet,1226.0,6.214858,46.199271


In [3]:
pip install haversine

Note: you may need to restart the kernel to use updated packages.


In [2]:
import haversine as hs   
from haversine import Unit
 
def calculate_distance (lat1, lon1, lat2, lon2):
    loc1=(lat1, lon1)
    loc2=(lat2, lon2)
 
    distance=hs.haversine(loc1,loc2,unit=Unit.METERS)
    return distance

location1 = [47.373032740890785, 8.498880746229132]
location2 = [47.370284238918465, 8.509635802377217]

dist = calculate_distance(location1[0], location1[1], location2[0], location2[1])
dist




865.6616515214281

<h3>Fetch all supermarktets in Switzerland with overpass turbo and save in df<h3/>

In [2]:
import json
import requests
import pandas as pd


# Request all restaurants, cafes,  in the Kanton of Zurich, Switzerland
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-1"="CH"][admin_level=2]->.a;
(node["shop"="supermarket"](area.a););
out center;
"""


# Server response
response = requests.get(overpass_url, params={'data': overpass_query})

data = response.json()

# Extract relevant data from the JSON response
places = []
for element in data['elements']:
    if 'tags' in element:
        place_type = element['tags'].get('amenity') or element['tags'].get('shop')
        places.append({
            'id': element['id'],
            'type': place_type,
            'lat': element['lat'],
            'lon': element['lon'],
            'name': element['tags'].get('name', ''),
            'brand': element['tags'].get('brand', ''),
            'address_city': element['tags'].get('addr:city', ''),
            'address_zip': element['tags'].get('addr:postcode', ''),
            'address_street': element['tags'].get('addr:street', ''),
            'address_number': element['tags'].get('addr:housenumber', ''),
            

        })

# Convert the list of dictionaries to a pandas DataFrame
supermarket_df = pd.DataFrame(places)

# Display the DataFrame
supermarket_df

,id,type,lat,lon,name,brand,address_city,address_zip,address_street,address_number
0,33126515,supermarket,47.155616,9.037915,Spar,Spar,,,,
1,36726161,supermarket,47.226191,8.980329,Migros,Migros,Uznach,8730,Zürcherstrasse,25
2,39768209,supermarket,47.225154,8.969868,Coop,Coop,Uznach,8730,,
3,39947904,supermarket,47.376732,8.542161,Coop,Coop,Zürich,8001,Bahnhofbrücke,1
4,48932835,supermarket,47.375020,8.522895,Migros,Migros,Zürich,8004,Wengistrasse,7
...,...,...,...,...,...,...,...,...,...,...
3416,11816025922,supermarket,47.334994,8.770158,M-Outlet,Outlet Migros,Aathal-Seegräben,8607,Zürichstrasse,15
3417,11816257943,supermarket,47.070895,8.284011,Aldi,ALDI,,,,
3418,11816339806,supermarket,47.507064,8.542320,Coop,Coop,Bachenbülach,8184,Grabenstrasse,6
3419,11829990637,supermarket,47.109750,7.391034,Volg,,Schnottwil,3253,Bernstrasse,17


<h3>Fetch all restaurants & cafes in Switzerland with overpass turbo and save in df<h3/>

In [3]:
import json
import requests
import pandas as pd


# Request all restaurants, cafes,  in the Kanton of Zurich, Switzerland
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-1"="CH"][admin_level=2]->.a;
(node["amenity"="restaurant"](area.a);
 node["amenity"="cafe"](area.a););
out center;
"""


# Server response
response = requests.get(overpass_url, params={'data': overpass_query})

data = response.json()

# Extract relevant data from the JSON response
places = []
for element in data['elements']:
    if 'tags' in element:
        place_type = element['tags'].get('amenity') or element['tags'].get('shop')
        places.append({
            'id': element['id'],
            'type': place_type,
            'lat': element['lat'],
            'lon': element['lon'],
            'name': element['tags'].get('name', ''),
            'brand': element['tags'].get('brand', ''),
            'address_city': element['tags'].get('addr:city', ''),
            'address_zip': element['tags'].get('addr:postcode', ''),
            'address_street': element['tags'].get('addr:street', ''),
            'address_number': element['tags'].get('addr:housenumber', ''),
            

        })

# Convert the list of dictionaries to a pandas DataFrame
foodandbeverage_df = pd.DataFrame(places)

# Display the DataFrame
foodandbeverage_df


,id,type,lat,lon,name,brand,address_city,address_zip,address_street,address_number
0,8204263,cafe,46.577343,8.387607,Rhonegletscher,,,,,
1,18476816,restaurant,47.044597,7.505473,Gourmet Party Service,,,3303,Bernstrasse,39
2,24977789,restaurant,46.774539,8.643673,Arnisee,,Intschi,6476,Gotthardstrasse,
3,24977794,restaurant,46.881010,8.645003,Lehnhof,,Altdorf UR,6460,Lehnplatz,18
4,24977799,restaurant,46.907817,8.624485,Apertura,,Flüelen,6454,Seestrasse,7
...,...,...,...,...,...,...,...,...,...,...
17560,11835931569,restaurant,47.046911,9.441821,Pizzeria Fantasy,,,,Bahnhofstrasse,9
17561,11835944869,restaurant,47.040299,9.438852,Bistro Pizzeria Riet,,,,Wolfrietstrasse,4
17562,11835967769,restaurant,47.047835,9.440107,Schützengarten,,,,,
17563,11835988764,restaurant,47.345160,7.903068,Kleinholz,,,,,


<h3>Test number of points of interest (e.g. supermarkets) around a single property<h3/>

In [5]:
import pandas as pd
import haversine as hs
from haversine import Unit

def find_nearby_poi(location_lat, location_lon, poi, radius):
    nearby_poi = pd.DataFrame(columns=poi.columns)  # Initialize an empty DataFrame to store nearby points of interest
    for index, point in poi.iterrows():
        # Calculate distance between location and point of interest
        distance = hs.haversine((location_lat, location_lon), 
                                (point['lat'], point['lon']), 
                                unit=Unit.METERS)
        if distance <= radius:
            # Add the point of interest to the nearby_poi DataFrame
            nearby_poi.loc[len(nearby_poi)] = point
    return nearby_poi

def count_nearby_poi(location_lat, location_lon, poi, radius):
    nearby_poi = find_nearby_poi(location_lat, location_lon, poi, radius)
    count = len(nearby_poi)
    return count, nearby_poi

# Assuming location_lat and location_lon are the latitude and longitude of the location
# and supermarket_df is your DataFrame containing points of interest

# Define the radius for nearby points of interest
radius = 500  # 500 meters

apartment = [47.372881725214576, 8.49887991367827]

# Count nearby points of interest for the location
poi_count, nearby_poi_df = count_nearby_poi(apartment[0], apartment[1], foodandbeverage_df, radius)

# Print the count of nearby points of interest
print("Number of nearby points of interest:", poi_count)

# Print the DataFrame containing nearby points of interest
print("Nearby points of interest:")
print(nearby_poi_df)


Number of nearby points of interest: 3
Nearby points of interest:
           id  type        lat       lon                      name brand  \
0   864928701  cafe  47.377332  8.498589          Cafe Letzistübli         
1  2207042728  cafe  47.373774  8.503635                 Kafi Guet         
2  2329582473  cafe  47.374555  8.496355  Bode Bäckerei-Konditorei         

  address_city address_zip     address_street address_number  
0                                                             
1       Zürich        8055         Gutstrasse            152  
2       Zürich        8047  Fellenbergstrasse            228  


<h3>Count the number of POIs for all apartments and add the count as a new column<h3/>

In [8]:
import pandas as pd

filepath = '../TablesDB/Location_v3.csv'

apartments_df = pd.read_csv(filepath, delimiter=",")
#code to test only a subset of the data before running it on the entire dataset
#apartments_df = apartments_df[:10]
apartments_df

,LocationId,Street,ZIP,longitude,latitude
0,78753131-6d78-4d1e-a0e5-fc8b5f430570,NaN,6598.0,8.849665,46.175962
1,f0c2bc0e-55ab-4eb1-98fa-edcb4a0ee01e,via albonago 43,6962.0,8.974113,46.013360
2,fbdcde66-0fd7-4304-a83e-70ecbb0f6ae7,Via San Gottardo 41,6500.0,9.032918,46.201556
3,2d7451a3-d7e1-4dc7-babf-ab237f2c02e4,Via F. Zorzi 17,6500.0,9.015443,46.187900
4,2bde1434-f480-4d46-9b05-99e0c0659671,Via San Gottardo,6900.0,8.941940,46.015976
...,...,...,...,...,...
11347,2dba2402-d23f-4ab8-9bb6-702bbe28cb4e,Chemin de la Petite-Californie,1222.0,6.207831,46.242178
11348,daed4779-995c-4c34-85e7-4b269d8bf5cc,Chemin William-Barbey,1292.0,6.149006,46.241745
11349,faddd38c-e9fc-4f4d-bf09-a41818cc620f,Rue du Temple 5,1236.0,6.018499,46.174626
11350,84d332ee-e1a8-4b5f-a432-82819fffe330,Chemin de la Pierre-à-Bochet,1226.0,6.214858,46.199271


In [9]:
import pandas as pd
import haversine as hs
from haversine import Unit

def count_nearby_poi(apartment, poi, radius):
    count = 0
    for index, point in poi.iterrows():
        # Calculate distance between apartment and point of interest
        distance = hs.haversine((apartment['latitude'], apartment['longitude']), 
                                (point['lat'], point['lon']), 
                                unit=Unit.METERS)
        if distance <= radius:
            count += 1
    return count


# Define the radius for nearby points of interest
radius = 500  # 500 meters

# Create an empty list to store the count of nearby points of interest for each apartment
poi_counts_supermarket = []
poi_counts_foodandbeverage = []

# Iterate through each apartment
for index, apartment in apartments_df.iterrows():
    # Count nearby points of interest for the current apartment
    count_supermarket = count_nearby_poi(apartment, supermarket_df, radius)
    count_foodandbeverage = count_nearby_poi(apartment, foodandbeverage_df, radius)
    poi_counts_supermarket.append(count_supermarket)
    poi_counts_foodandbeverage.append(count_foodandbeverage)


# Add the list of counts to the apartments_df DataFrame as a new column
apartments_df['#supermarkets'] = poi_counts_supermarket
apartments_df['#foodandbeverage'] = poi_counts_foodandbeverage

# Show the DataFrame with the count of nearby points of interest for each apartment
apartments_df

,LocationId,Street,ZIP,longitude,latitude,#supermarkets,#foodandbeverage
0,78753131-6d78-4d1e-a0e5-fc8b5f430570,NaN,6598.0,8.849665,46.175962,2,3
1,f0c2bc0e-55ab-4eb1-98fa-edcb4a0ee01e,via albonago 43,6962.0,8.974113,46.013360,0,3
2,fbdcde66-0fd7-4304-a83e-70ecbb0f6ae7,Via San Gottardo 41,6500.0,9.032918,46.201556,0,1
3,2d7451a3-d7e1-4dc7-babf-ab237f2c02e4,Via F. Zorzi 17,6500.0,9.015443,46.187900,0,6
4,2bde1434-f480-4d46-9b05-99e0c0659671,Via San Gottardo,6900.0,8.941940,46.015976,3,3
...,...,...,...,...,...,...,...
11347,2dba2402-d23f-4ab8-9bb6-702bbe28cb4e,Chemin de la Petite-Californie,1222.0,6.207831,46.242178,0,0
11348,daed4779-995c-4c34-85e7-4b269d8bf5cc,Chemin William-Barbey,1292.0,6.149006,46.241745,0,0
11349,faddd38c-e9fc-4f4d-bf09-a41818cc620f,Rue du Temple 5,1236.0,6.018499,46.174626,0,1
11350,84d332ee-e1a8-4b5f-a432-82819fffe330,Chemin de la Pierre-à-Bochet,1226.0,6.214858,46.199271,0,0


<h3>Safe df to csv<h3/>

In [10]:
filepath_csv = '../TablesDB/Location_v3_enriched.csv'
apartments_df.to_csv(filepath_csv, index=False)

In [11]:
filepath_csv = '../TablesDB/Location_v3_enriched.csv'
check_csv = pd.read_csv(filepath_csv, sep=",")
check_csv

,LocationId,Street,ZIP,longitude,latitude,#supermarkets,#foodandbeverage
0,78753131-6d78-4d1e-a0e5-fc8b5f430570,NaN,6598.0,8.849665,46.175962,2,3
1,f0c2bc0e-55ab-4eb1-98fa-edcb4a0ee01e,via albonago 43,6962.0,8.974113,46.013360,0,3
2,fbdcde66-0fd7-4304-a83e-70ecbb0f6ae7,Via San Gottardo 41,6500.0,9.032918,46.201556,0,1
3,2d7451a3-d7e1-4dc7-babf-ab237f2c02e4,Via F. Zorzi 17,6500.0,9.015443,46.187900,0,6
4,2bde1434-f480-4d46-9b05-99e0c0659671,Via San Gottardo,6900.0,8.941940,46.015976,3,3
...,...,...,...,...,...,...,...
11347,2dba2402-d23f-4ab8-9bb6-702bbe28cb4e,Chemin de la Petite-Californie,1222.0,6.207831,46.242178,0,0
11348,daed4779-995c-4c34-85e7-4b269d8bf5cc,Chemin William-Barbey,1292.0,6.149006,46.241745,0,0
11349,faddd38c-e9fc-4f4d-bf09-a41818cc620f,Rue du Temple 5,1236.0,6.018499,46.174626,0,1
11350,84d332ee-e1a8-4b5f-a432-82819fffe330,Chemin de la Pierre-à-Bochet,1226.0,6.214858,46.199271,0,0


<h3>Similar code to above which also creates a list of all POIs that were counted. Attention, will also create a POI count column in the apartments_df.<h3/>

In [ ]:
import pandas as pd

filepath = '../TablesDB/Location_v3.csv'

apartments_df = pd.read_csv(filepath, delimiter=",")
#code to test only a subset of the data before running it on the entire dataset
apartments_df = apartments_df[:10]
apartments_df

,LocationId,Street,ZIP,longitude,latitude
0,78753131-6d78-4d1e-a0e5-fc8b5f430570,NaN,6598.0,8.849665,46.175962
1,f0c2bc0e-55ab-4eb1-98fa-edcb4a0ee01e,via albonago 43,6962.0,8.974113,46.013360
2,fbdcde66-0fd7-4304-a83e-70ecbb0f6ae7,Via San Gottardo 41,6500.0,9.032918,46.201556
3,2d7451a3-d7e1-4dc7-babf-ab237f2c02e4,Via F. Zorzi 17,6500.0,9.015443,46.187900
4,2bde1434-f480-4d46-9b05-99e0c0659671,Via San Gottardo,6900.0,8.941940,46.015976
...,...,...,...,...,...
11347,2dba2402-d23f-4ab8-9bb6-702bbe28cb4e,Chemin de la Petite-Californie,1222.0,6.207831,46.242178
11348,daed4779-995c-4c34-85e7-4b269d8bf5cc,Chemin William-Barbey,1292.0,6.149006,46.241745
11349,faddd38c-e9fc-4f4d-bf09-a41818cc620f,Rue du Temple 5,1236.0,6.018499,46.174626
11350,84d332ee-e1a8-4b5f-a432-82819fffe330,Chemin de la Pierre-à-Bochet,1226.0,6.214858,46.199271


In [12]:
import pandas as pd
import haversine as hs
from haversine import Unit

def find_nearby_poi(apartment, poi, radius):
    nearby_poi = pd.DataFrame(columns=poi.columns)  # Initialize an empty DataFrame to store nearby points of interest
    for index, point in poi.iterrows():
        # Calculate distance between apartment and point of interest
        distance = hs.haversine((apartment['latitude'], apartment['longitude']), 
                                (point['lat'], point['lon']), 
                                unit=Unit.METERS)
        if distance <= radius:
            # Add the point of interest to the nearby_poi DataFrame
            nearby_poi.loc[len(nearby_poi)] = point
    # Add LocationID to the DataFrame
    nearby_poi['LocationId'] = apartment['LocationId']
    return nearby_poi

def count_nearby_poi(apartment, poi, radius):
    nearby_poi = find_nearby_poi(apartment, poi, radius)
    count = len(nearby_poi)
    return count, nearby_poi


# Define the radius for nearby points of interest
radius = 500  # 500 meters

# Create an empty list to store the count of nearby points of interest for each apartment
poi_counts = []
poi_dfs = []

# Iterate through each apartment
for index, apartment in apartments_df.iterrows():
    # Count nearby points of interest for the current apartment
    count, nearby_poi_df = count_nearby_poi(apartment, supermarket_df, radius)
    poi_counts.append(count)
    poi_dfs.append(nearby_poi_df)

# Add the list of counts to the apartments_df DataFrame as a new column
apartments_df['supermarket_count'] = poi_counts

# Concatenate the list of DataFrames containing nearby supermarkets
nearby_supermarkets_df = pd.concat(poi_dfs, ignore_index=True)




In [13]:
apartments_df

,LocationId,Street,ZIP,longitude,latitude,#supermarkets,supermarket_count
0,78753131-6d78-4d1e-a0e5-fc8b5f430570,NaN,6598.0,8.849665,46.175962,2,2
1,f0c2bc0e-55ab-4eb1-98fa-edcb4a0ee01e,via albonago 43,6962.0,8.974113,46.013360,0,0
2,fbdcde66-0fd7-4304-a83e-70ecbb0f6ae7,Via San Gottardo 41,6500.0,8.808332,46.174824,3,3
3,2d7451a3-d7e1-4dc7-babf-ab237f2c02e4,Via F. Zorzi 17,6500.0,9.015443,46.187900,0,0
4,2bde1434-f480-4d46-9b05-99e0c0659671,Via San Gottardo,6900.0,8.840817,46.178948,0,0
5,9ef1dca5-9f48-4aa3-9a3f-91e653dd54e1,strada Növa 3,6968.0,8.989980,46.058053,1,1
6,8471d639-cef7-4700-be5b-65768d8aafea,NaN,6922.0,8.910016,45.935817,0,0
7,ec03f7bb-74d4-4eb3-b59a-e972bf84ef2f,Via Maestri Comacini 29,6834.0,9.013579,46.181216,3,3
8,dc822535-9c98-40fb-8892-0a038280899d,via novel 7,6967.0,8.227512,46.818188,0,0
9,11f9283c-ed92-427e-865c-66f211ad77c7,Viale Attilio Tarchini 9B,6828.0,9.003807,45.845475,0,0


In [14]:
nearby_supermarkets_df

,id,type,lat,lon,name,brand,address_city,address_zip,address_street,address_number,LocationId
0,723124727,supermarket,46.175037,8.854861,Coop,Coop,,,,,78753131-6d78-4d1e-a0e5-fc8b5f430570
1,723124737,supermarket,46.177868,8.85332,Denner,Denner,,,,,78753131-6d78-4d1e-a0e5-fc8b5f430570
2,805177065,supermarket,46.175257,8.811153,Migros,Migros,,,Via San Gottardo,,fbdcde66-0fd7-4304-a83e-70ecbb0f6ae7
3,805177095,supermarket,46.175114,8.812309,Coop,Coop,,,Via Municipio,,fbdcde66-0fd7-4304-a83e-70ecbb0f6ae7
4,805177100,supermarket,46.175406,8.810065,Denner,Denner,,,,,fbdcde66-0fd7-4304-a83e-70ecbb0f6ae7
5,417175664,supermarket,46.057978,8.989863,Crai,,Sonvico,6968,Stráda dra Pésa,2,9ef1dca5-9f48-4aa3-9a3f-91e653dd54e1
6,1750147269,supermarket,46.176832,9.014709,Denner,Denner,,,,,ec03f7bb-74d4-4eb3-b59a-e972bf84ef2f
7,2548875263,supermarket,46.178328,9.013883,Migros,Migros,,,,,ec03f7bb-74d4-4eb3-b59a-e972bf84ef2f
8,4035866381,supermarket,46.1819,9.018093,Lidl,Lidl,,,,,ec03f7bb-74d4-4eb3-b59a-e972bf84ef2f
